## 1. MODIS VCH Collection 6 masks, 6.1 experiments

This notebook is to generate 3-layer mod44b masks given a different threshold.

Kernel: ILAB Kernel (PyTorch)

Caleb Spradlin 08/07/2024

## 2. Configuration

In [1]:
tile = 'h11v02'

threshold = 80
comparison = '>=' # '<=' or '>='

maskValue = 100 # Don't worry about this (for now)
replacementValue = 100

statistic = 'med' # 'med', min', or 'max'
year = 2019 # Keep at 2019

subdataset = 'Percent_NonVegetated' # 'Percent_Tree_Cover'
outputDir = '/explore/nobackup/people/cssprad1/projects/modis_vcf/data/testing/' # Change to your own output dir

## 3. Code section (skip to 6 for resulting path)

In [2]:
import datetime
import glob
import logging
import os
import time
from pathlib import Path

import numpy as np
import pandas as pd
import rasterio as rio
from osgeo import gdal, gdal_array
from osgeo.osr import SpatialReference

os.environ['USE_PYGEOS'] = '0'
os.umask(0o022)

63

In [3]:
# Create a logger object
logger = logging.getLogger('simple_logger')
logger.setLevel(logging.DEBUG)  # Set the logging level

# Create a console handler to log messages to stdout
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)  # Log all levels to the console

# Create a simple formatter
formatter = logging.Formatter('%(levelname)s - %(message)s')
console_handler.setFormatter(formatter)

# Add the handler to the logger
logger.addHandler(console_handler)

In [4]:
def getFilePath(directory: Path, searchPattern: str) -> Path:
    """Retrieve the first file path that matches the search pattern in the specified directory."""
    files = sorted(directory.glob(searchPattern))
    if not files:
        raise FileNotFoundError(f"No files found matching {searchPattern}")
    if len(files) > 1:
        logging.warning(f"Multiple files found for pattern {searchPattern}, using {files[0]}")
    return files[0]

def makeMod44bMask(tile: str, threshold: float, maskDir: Path, outDir: Path,
                   subdataset: str, comparison: str = '>=', statistic: str = 'min',
                   maskValue: int = 1) -> str:
    """
    Generate a mask for a MOD44B dataset by comparing a specified statistic (min, median, max) 
    across rasters against a threshold, and save the result as a GeoTIFF.
    """
    os.makedirs(outDir, exist_ok=True)
    
    # Print the tile being processed
    print(f'TILE: {tile}')
    
    # Find the raster files matching the tile
    currTiles = sorted(list(maskDir.glob(f'*{tile}.tif')))
    print(f'Raster stack: {currTiles}')
    
    # Read all rasters into a list of arrays
    arrays = []
    with rio.open(currTiles[0]) as src:
        profile = src.profile
        transform = src.transform
        crs = src.crs
    
    for raster in currTiles:
        with rio.open(raster) as src:
            arrays.append(src.read(1))  # Read the first band of each raster

    # Convert list of arrays to a 3D numpy array (stacking along axis=0)
    arrayStack = np.stack(arrays, axis=0)
    print(f'Array stack shape: {arrayStack.shape}')
    
    # Calculate the specified statistic across the first axis (axis=0)
    if statistic == 'min':
        values = np.min(arrayStack, axis=0)
    elif statistic == 'med':
        values = np.median(arrayStack, axis=0)
    elif statistic == 'max':
        values = np.max(arrayStack, axis=0)
    else:
        raise ValueError("Invalid statistic. Use 'min', 'med', or 'max'.")

    # Determine the comparison operation
    if comparison == '>=':
        resultArr = np.where(values >= threshold, maskValue, 0)
    elif comparison == '==':
        resultArr = np.where(values == threshold, maskValue, 0)
    elif comparison == '<=':
        resultArr = np.where(values <= threshold, maskValue, 0)
    else:
        raise ValueError("Invalid comparison operator. Use '>=', '==', or '<='.")
    
    # Update the metadata to match the new array
    profile.update({
        "driver": "GTiff",
        "height": resultArr.shape[0],
        "width": resultArr.shape[1],
        "count": 1,  # Number of bands
        "dtype": 'uint8',  # Binary data type
        "nodata": 0  # NoData value should be 0 for binary arrays
    })

    # Write the new array to a new raster using the updated metadata
    resultRasterPath = os.path.join(outDir, f'{tile}_{statistic}_{subdataset}.tif')
    
    with rio.open(resultRasterPath, 'w', **profile) as dst:
        dst.write(resultArr, 1)

    print(f"New raster written to {resultRasterPath}")
    
    return resultRasterPath

def openHdf(hdfPath: str, subdatasetIndex: int):
    """
    Open an HDF file and retrieve a specified subdataset along with its array, projection, 
    and geotransform information.
    """
    hdf = gdal.Open(hdfPath)
    subdatasetPath = hdf.GetSubDatasets()[subdatasetIndex][0]
    
    print(f'Opening {subdatasetPath}')
    
    subdataset = gdal.Open(subdatasetPath)
    projection = subdataset.GetProjection()
    geotransform = subdataset.GetGeoTransform()
    subdatasetArray = subdataset.ReadAsArray()
    
    return subdatasetArray, subdatasetPath, projection, geotransform

def generateFillValueMask(treeArray, nonVegArray):
    
    condition = ((treeArray == 253) | (nonVegArray == 253))
    
    return np.where(condition, 1, 0)

def generateOutOfProjectionMask(waterMaskArray: np.ndarray) -> np.ndarray:
    """Generate a mask for out-of-projection pixels (value 250 in MOD44W dataset)."""
    return np.where(waterMaskArray == 250, 1, 0)

def generateWaterMask(waterMaskArray: np.ndarray) -> np.ndarray:
    """Generate a water mask for pixels with value 1 in the MOD44W dataset."""
    return np.where(waterMaskArray == 1, 1, 0)

def getMod44wMasks(mod44wHdfPath: str):
    """
    Retrieve out-of-projection and water masks from a MOD44W HDF file.
    """
    waterMaskSubdatasetIdx = 0
    waterMaskArray, _, _, _ = openHdf(mod44wHdfPath, waterMaskSubdatasetIdx)

    oopMask = generateOutOfProjectionMask(waterMaskArray)
    waterMask = generateWaterMask(waterMaskArray)
    
    return oopMask, waterMask

def readMask(maskPath: Path, dtype=np.int16) -> np.ndarray:
    """Read a mask file into a numpy array with the specified data type."""
    ds = gdal.Open(str(maskPath), gdal.GA_ReadOnly)
    if ds is None:
        raise FileNotFoundError(f"Could not open mask file {maskPath}")
    maskArray = ds.ReadAsArray().astype(dtype)
    return maskArray

def applyMaskAndSaveToGeoTiff(hdfPath: Path, subdataset: str, maskPath: Path, 
                              maskValue: int, replacementValue: int, 
                              waterMaskHdfPath: Path, outputPath: Path) -> None:
    """
    Apply a mask to a subdataset from an HDF file, combine it with out-of-projection and water masks, 
    and save the result as a GeoTIFF.
    """
    oopMask, waterMask = getMod44wMasks(str(waterMaskHdfPath))    

    ds = gdal.Open(str(hdfPath), gdal.GA_ReadOnly)
    if ds is None:
        raise FileNotFoundError(f"Could not open HDF file {hdfPath}")
    
    subdatasets = ds.GetSubDatasets()
    subdatasetNames = [subdataset[0].split(':')[-1] for subdataset in subdatasets]
    subdatasetIdx = subdatasetNames.index(subdataset)
    subdatasetPath = subdatasets[subdatasetIdx][0]

    subds = gdal.Open(subdatasetPath, gdal.GA_ReadOnly)
    if subds is None:
        raise FileNotFoundError(f"Could not open subdataset {subdataset}")
    
    dataArray = subds.ReadAsArray()

    print(f'Masking with {maskPath}')
    maskArray = readMask(maskPath)
    numberPixelMasked = np.count_nonzero((maskArray == maskValue))
    print(f'Masking {numberPixelMasked} pixels to {replacementValue}')
    maskedArray = np.where(maskArray == maskValue, replacementValue, dataArray)

    waterValue = 200
    oopValue = 250
    
    print(f'Masking out {np.count_nonzero(oopMask)} out-of-projection pixels')
    maskedArray = np.where(oopMask, oopValue, maskedArray)
    
    print(f'Masking out {np.count_nonzero(waterMask)} water pixels')
    maskedArray = np.where(waterMask, waterValue, maskedArray)

    # Write to GeoTIFF
    driver = gdal.GetDriverByName('GTiff')
    if driver is None:
        raise RuntimeError("GeoTIFF driver is not available")
    
    outDs = driver.Create(str(outputPath), maskedArray.shape[1], maskedArray.shape[0], 1, gdal.GDT_Int16)
    if outDs is None:
        raise RuntimeError(f"Could not create GeoTIFF file {outputPath}")

    outDs.GetRasterBand(1).WriteArray(maskedArray)
    print(f'Wrote masked subdataset to {outputPath}')
    
    # Copy projection and geotransform from the original dataset
    outDs.SetProjection(ds.GetProjection())
    outDs.SetGeoTransform(ds.GetGeoTransform())
    
    return outputPath

def generateThreeLayerData(nonVegPath, percentTreeHdfPath, mod44wHdfPath, outputDir):
    
    percentTreeSubdatasetIdx = 0
    treeArray, _, projection, geotransform = openHdf(
        str(percentTreeHdfPath), percentTreeSubdatasetIdx)

    waterMaskSubdatasetIdx = 0
    waterMaskArray, _, _, _ = openHdf(
        str(mod44wHdfPath), waterMaskSubdatasetIdx)

    with rio.open(nonVegPath) as src:
        nonVegArray = src.read(1)  # Read the first band of each raster

    # Get fill value mask 
    fillMask = generateFillValueMask(treeArray, nonVegArray)

    # Get OOP mask
    oopMask = generateOutOfProjectionMask(waterMaskArray)
    
    # Get water mask
    waterMask = generateWaterMask(waterMaskArray)

    # Create non-tree vegetation layer
    nonTreeVeg = np.zeros(waterMask.shape, dtype=np.int16)
    nonTreeVeg.fill(100)
    nonTreeVeg = nonTreeVeg - nonVegArray
    nonTreeVeg = nonTreeVeg - treeArray
    
    fillValue = 253
    waterValue = 200
    oopValue = 250

    # Mask fill first
    treeArray = np.where(fillMask, fillValue, treeArray)
    nonVegArray = np.where(fillMask, fillValue, nonVegArray)
    nonTreeVeg = np.where(fillMask, fillValue, nonTreeVeg)
    
    # Then water
    treeArray = np.where(waterMask, waterValue, treeArray)
    nonVegArray = np.where(waterMask, waterValue, nonVegArray)
    nonTreeVeg = np.where(waterMask, waterValue, nonTreeVeg)
    
    # Then out-of-projection
    treeArray = np.where(oopMask, oopValue, treeArray)
    nonVegArray = np.where(oopMask, oopValue, nonVegArray)
    nonTreeVeg = np.where(oopMask, oopValue, nonTreeVeg)

    nonTreeVeg = np.where(nonTreeVeg < 0, fillValue, nonTreeVeg)
    nonTreeVeg = nonTreeVeg.astype(np.uint8)
    
    nonTreeVeg = np.where(((treeArray == waterValue) | (nonVegArray == waterValue)), waterValue, nonTreeVeg)
    nonTreeVeg = np.where(((treeArray == fillValue) | (nonVegArray == fillValue)), fillValue, nonTreeVeg)
    
    treeArray = np.where(nonTreeVeg > 100, nonTreeVeg, treeArray)
    nonVegArray = np.where(nonTreeVeg > 100, nonTreeVeg, nonVegArray)

    preStrName = os.path.basename(percentTreeHdfPath).split('.')[:4]
    preStrName.extend(['Median', 'Masked', 'tif'])
    outputName = '.'.join(preStrName)
    outputPath = os.path.join(outputDir, outputName)

    driver = gdal.GetDriverByName('GTiff')
    dataset = driver.Create(outputPath, nonTreeVeg.shape[1], nonTreeVeg.shape[0], 3, gdal.GDT_Byte)

    # Set projection and geotransform
    dataset.SetProjection(projection)
    dataset.SetGeoTransform(geotransform)

    # Write each band to the file
    dataset.GetRasterBand(1).WriteArray(treeArray)
    dataset.GetRasterBand(2).WriteArray(nonTreeVeg)
    dataset.GetRasterBand(3).WriteArray(nonVegArray)

    # Set band descriptions
    dataset.GetRasterBand(1).SetDescription('Percent_Tree_Cover MOD44B_250m_GRID (8-bit unsigned integer) Median Masked')
    dataset.GetRasterBand(2).SetDescription('Percent_NonTree_Vegetation MOD44B_250m_GRID (8-bit unsigned integer) Median Masked')
    dataset.GetRasterBand(3).SetDescription('Percent_NonVegetated MOD44B_250m_GRID (8-bit unsigned integer) Median Masked')

    # Close the dataset
    dataset = None

    print(f"GeoTIFF file '{outputPath}' created successfully.")
    
    return outputPath

## 4. Paths determined from configuration

In [5]:
mask_post_str = f'{statistic}_{subdataset}.tif'

waterMaskHdfDir: Path = Path('/css/modis/Collection6.1/L3/MOD44W-LandWaterMask/2019/001')
waterMaskHdfPath = getFilePath(waterMaskHdfDir, f'*{tile}*.hdf')
print(f'Water mask: {waterMaskHdfPath}\n')

hdfDir = Path('/explore/nobackup/people/mfrost2/temp/MOD44B_1978/MOD44B/2019/065')
hdfPath = getFilePath(hdfDir, f'*{tile}*.hdf')
print(f'MOD44B HDF: {hdfPath}\n')

outDir =  Path(outputDir)
maskDir = Path(f'/explore/nobackup/people/cssprad1/projects/modis_vcf/data/masks/{subdataset}')
print(f'Mask directory: {maskDir}\n')
# mask_dir = Path('/explore/nobackup/people/cssprad1/projects/modis_vcf/data/Coll6_mask_stacks')

maskedHdfTifName = f'{tile}_{year}_{subdataset}_{statistic}_masked.tif'
maskedHdfTifPath = outDir / maskedHdfTifName
print(f'Masked HDF subdataset path {maskedHdfTifPath}\n')

medianPercentTreeHdf = sorted(list(Path('/explore/nobackup/people/cssprad1/projects/modis_vcf/development/tree_med').glob(f'*{tile}*.hdf')))[0]
print(f'Median Percent Tree Cover: {medianPercentTreeHdf}')

Water mask: /css/modis/Collection6.1/L3/MOD44W-LandWaterMask/2019/001/MOD44W.A2019001.h11v02.061.2024007232245.hdf

MOD44B HDF: /explore/nobackup/people/mfrost2/temp/MOD44B_1978/MOD44B/2019/065/MOD44B.A2019065.h11v02.061.2024156162738.hdf

Mask directory: /explore/nobackup/people/cssprad1/projects/modis_vcf/data/masks/Percent_NonVegetated

Masked HDF subdataset path /explore/nobackup/people/cssprad1/projects/modis_vcf/data/testing/h11v02_2019_Percent_NonVegetated_med_masked.tif

Median Percent Tree Cover: /explore/nobackup/people/cssprad1/projects/modis_vcf/development/tree_med/MOD44B.A2019065.h11v02.061.2024212170815.hdf


## 5. Generate masks

In [6]:
maskRasterPath = makeMod44bMask(tile,
                                  threshold=threshold,
                                  maskDir=maskDir,
                                  outDir=outDir,
                                  subdataset=subdataset,
                                  comparison=comparison,
                                  statistic=statistic,
                                  maskValue=maskValue)

TILE: h11v02
Raster stack: [PosixPath('/explore/nobackup/people/cssprad1/projects/modis_vcf/data/masks/Percent_NonVegetated/2000.006.h11v02.tif'), PosixPath('/explore/nobackup/people/cssprad1/projects/modis_vcf/data/masks/Percent_NonVegetated/2001.006.h11v02.tif'), PosixPath('/explore/nobackup/people/cssprad1/projects/modis_vcf/data/masks/Percent_NonVegetated/2002.006.h11v02.tif'), PosixPath('/explore/nobackup/people/cssprad1/projects/modis_vcf/data/masks/Percent_NonVegetated/2003.006.h11v02.tif'), PosixPath('/explore/nobackup/people/cssprad1/projects/modis_vcf/data/masks/Percent_NonVegetated/2004.006.h11v02.tif'), PosixPath('/explore/nobackup/people/cssprad1/projects/modis_vcf/data/masks/Percent_NonVegetated/2005.006.h11v02.tif'), PosixPath('/explore/nobackup/people/cssprad1/projects/modis_vcf/data/masks/Percent_NonVegetated/2006.006.h11v02.tif'), PosixPath('/explore/nobackup/people/cssprad1/projects/modis_vcf/data/masks/Percent_NonVegetated/2007.006.h11v02.tif'), PosixPath('/explore/

## 6. Generate updated HDF subdataset 3-layer TIF

In [7]:
maskedTifPath = applyMaskAndSaveToGeoTiff(hdfPath=hdfPath,
                                               subdataset=subdataset,
                                               maskPath=maskRasterPath,
                                               maskValue=maskValue,
                                               replacementValue=replacementValue,
                                               waterMaskHdfPath=waterMaskHdfPath,
                                               outputPath=maskedHdfTifPath)

print('\n\n###############')
print(f'Condition (array {comparison} {threshold})')
print(maskedHdfTifPath)
print('###############')

Opening HDF4_EOS:EOS_GRID:"/css/modis/Collection6.1/L3/MOD44W-LandWaterMask/2019/001/MOD44W.A2019001.h11v02.061.2024007232245.hdf":MOD44W_250m_GRID:water_mask
Masking with /explore/nobackup/people/cssprad1/projects/modis_vcf/data/testing/h11v02_med_Percent_NonVegetated.tif
Masking 8013605 pixels to 100
Masking out 2794969 out-of-projection pixels
Masking out 4012741 water pixels
Wrote masked subdataset to /explore/nobackup/people/cssprad1/projects/modis_vcf/data/testing/h11v02_2019_Percent_NonVegetated_med_masked.tif


###############
Condition (array >= 80)
/explore/nobackup/people/cssprad1/projects/modis_vcf/data/testing/h11v02_2019_Percent_NonVegetated_med_masked.tif
###############


In [8]:
threeLayerData = generateThreeLayerData(maskedTifPath, medianPercentTreeHdf, waterMaskHdfPath, outDir)

Opening HDF4_EOS:EOS_GRID:"/explore/nobackup/people/cssprad1/projects/modis_vcf/development/tree_med/MOD44B.A2019065.h11v02.061.2024212170815.hdf":MOD44B_250m_GRID:Percent_Tree_Cover
Opening HDF4_EOS:EOS_GRID:"/css/modis/Collection6.1/L3/MOD44W-LandWaterMask/2019/001/MOD44W.A2019001.h11v02.061.2024007232245.hdf":MOD44W_250m_GRID:water_mask
GeoTIFF file '/explore/nobackup/people/cssprad1/projects/modis_vcf/data/testing/MOD44B.A2019065.h11v02.061.Median.Masked.tif' created successfully.
